<a href="https://colab.research.google.com/github/itzpankajpanwar/FakeNewsPropagation/blob/main/GCN_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages.
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric
!pip install vaderSentiment

     |████████████████████████████████| 2.6MB 4.6MB/s 
     |████████████████████████████████| 1.5MB 4.3MB/s 
     |████████████████████████████████| 215kB 6.6MB/s 
     |████████████████████████████████| 235kB 49.6MB/s 
     |████████████████████████████████| 2.2MB 45.3MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 133kB 6.2MB/s 


In [ ]:
!git clone https://github.com/itzpankajpanwar/FakeNewsPropagation.git

Cloning into 'FakeNewsPropagation'...
remote: Enumerating objects: 11383, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 11383 (delta 25), reused 0 (delta 0), pack-reused 11337
Receiving objects: 100% (11383/11383), 149.79 MiB | 23.46 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Checking out files: 100% (11295/11295), done.


In [ ]:
import sys
sys.path.insert(1,"/content/FakeNewsPropagation")
from  construct_sample_features import get_TPNF_dataset
sample_feature_arr = get_TPNF_dataset("data/features", "gossipcop", True, True, True,
                                        True, False, None, False)
print("\nsample array shape is : ",sample_feature_arr.shape)

(['Micro - Tree depth', 'Micro - No of nodes', 'Micro - Maximum out degree', 'No. of cascades with replies', 'Fraction of cascades with replies', 'Macro - Tree depth', 'Macro - No of nodes', 'Macro - Maximum out degree', 'Macro - No of cascades', "Macro - Max out degree node's level", 'No. of cascades with retweets', 'Fraction of cascades with retweets', 'No. of bot users retweeting', 'Fraction of bot user retweeting'], ['S10', 'S11', 'S12', 'S13', 'S14', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9'])
Feature group : Structural
3684
3684
(['Average time diff between adjacent replies', 'Time diff between first tweet posting node and first reply node', 'Time diff between first tweet posting news and last reply node', 'Average time between adjacent reply nodes in the deepest cascade', 'Time diff between first tweet posting news and last reply node in the deepest cascade', 'Average time diff between the adjacent retweet nodes in macro network', 'Time diff between first tweet and  m

In [ ]:
import torch
from torch_geometric.data import Data
import networkx as nx
import json
import os
from networkx.readwrite import json_graph
from load_dataset import get_dataset_sample_ids
import numpy as np


news_sourc = ["gossipcop"]

data_list = []
tt=0
for news_source in news_sourc:
    dataset_dir = "/content/FakeNewsPropagation/data/nx_network_data/nx_network_data"
    labels = ["fake","real"]

    
    for  news_label in labels:
        news_dataset_dir = "{}/{}_{}".format(dataset_dir, news_source, news_label)
        for sample_id in get_dataset_sample_ids(news_source, news_label, "/content/FakeNewsPropagation/data/sample_ids"):
            with open("{}/{}.json".format(news_dataset_dir, sample_id)) as file:
                new_graph = json_graph.tree_graph(json.load(file))
                i=0
                visited={}
                edge_list = new_graph.edges()
                edge_index = []
                for x , y in edge_list:
                  if x in visited:
                    if y in visited:
                      edge_index.append([visited[x],visited[y]])
                    else:
                       edge_index.append([visited[x],i])
                       visited[y]=i
                       i=i+1
                  else: 
                     if y in visited:
                      edge_index.append([i,visited[y]])
                      visited[x]=i
                      i=i+1
                     else:
                       edge_index.append([i,i+1])    
                       visited[x]=i
                       visited[y]=i+1
                       i=i+2           
                  #print(visited[x],"  ",visited[y],"\n")
                nodefeature=[]
                deg_centrality = nx.degree_centrality(new_graph)
                clos_centrality = nx.closeness_centrality(new_graph)
                page_centrality = nx.pagerank(new_graph,alpha=0.8)
                for x in deg_centrality:
                  trt=sample_feature_arr[tt]
                  at=np.array([])
                  at = np.append(at,deg_centrality[x])
                  at = np.append(at,clos_centrality[x])
                  at = np.append(at,page_centrality[x])
                  nodefeature.append(np.append(trt, at))
            
                tt+=1
              
                edge_index = torch.from_numpy(np.array(edge_index, dtype=np.long))
                x = torch.from_numpy(np.array(nodefeature, dtype=np.long))
                if news_label == "fake":
                  #for _ in range(nd):
                  y=0
                else:
                  #for _ in range(nd):
                  y=1
                data = Data(x=x, edge_index=edge_index.t().contiguous(),y=y)
                data_list.append(data)
print("length of data is : ",len(data_list))

length of data is :  7368


In [ ]:
from torch_geometric.data import Data, DataLoader
from torch_sparse import SparseTensor
loader = DataLoader(data_list)

#dataset = loader.dataset.shuffle()

train_size = int(0.8 * len(loader.dataset))
test_size = len(loader.dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(loader.dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, num_workers=0, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=0, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 32
Batch(batch=[11724], edge_index=[2, 11692], ptr=[33], x=[11724, 30], y=[32])

Step 2:
Number of graphs in the current batch: 32
Batch(batch=[3209], edge_index=[2, 3177], ptr=[33], x=[3209, 30], y=[32])

Step 3:
Number of graphs in the current batch: 32
Batch(batch=[5107], edge_index=[2, 5075], ptr=[33], x=[5107, 30], y=[32])

Step 4:
Number of graphs in the current batch: 32
Batch(batch=[2218], edge_index=[2, 2186], ptr=[33], x=[2218, 30], y=[32])

Step 5:
Number of graphs in the current batch: 32
Batch(batch=[10479], edge_index=[2, 10447], ptr=[33], x=[10479, 30], y=[32])

Step 6:
Number of graphs in the current batch: 32
Batch(batch=[8846], edge_index=[2, 8814], ptr=[33], x=[8846, 30], y=[32])

Step 7:
Number of graphs in the current batch: 32
Batch(batch=[4126], edge_index=[2, 4094], ptr=[33], x=[4126, 30], y=[32])

Step 8:
Number of graphs in the current batch: 32
Batch(batch=[6564], edge_index=[2, 6532], ptr=[33], x=[6564, 30], y=[

In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GraphConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(30, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.conv4 = GraphConv(hidden_channels, hidden_channels)
        self.conv5 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
        x = x.relu()
        x = self.conv5(x, edge_index)


        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GraphConv(30, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (conv4): GraphConv(64, 64)
  (conv5): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.

         data.x=data.x.float()
         for a in data.edge_index:
          a[0]=a[0].float()
          a[1]=a[1].float()
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         #print("data y is  : ",data.y)
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         data.x=data.x.float()
         for a in data.edge_index:
          a[0]=a[0].float()
          a[1]=a[1].float()
         out = model(data.x, data.edge_index, data.batch)  
         #print("output shape : ",out.shape)
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 31):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5215, Test Acc: 0.5251
Epoch: 002, Train Acc: 0.7012, Test Acc: 0.6995
Epoch: 003, Train Acc: 0.6714, Test Acc: 0.6689
Epoch: 004, Train Acc: 0.5896, Test Acc: 0.5902
Epoch: 005, Train Acc: 0.6714, Test Acc: 0.6703
Epoch: 006, Train Acc: 0.6773, Test Acc: 0.6852
Epoch: 007, Train Acc: 0.5606, Test Acc: 0.5563
Epoch: 008, Train Acc: 0.6233, Test Acc: 0.6377
Epoch: 009, Train Acc: 0.7033, Test Acc: 0.7083
Epoch: 010, Train Acc: 0.6909, Test Acc: 0.7042
Epoch: 011, Train Acc: 0.6401, Test Acc: 0.6588
Epoch: 012, Train Acc: 0.7202, Test Acc: 0.7395
Epoch: 013, Train Acc: 0.7160, Test Acc: 0.7259
Epoch: 014, Train Acc: 0.7153, Test Acc: 0.7198
Epoch: 015, Train Acc: 0.6680, Test Acc: 0.6744
Epoch: 016, Train Acc: 0.6089, Test Acc: 0.6133
Epoch: 017, Train Acc: 0.7253, Test Acc: 0.7341
Epoch: 018, Train Acc: 0.6254, Test Acc: 0.6357
Epoch: 019, Train Acc: 0.7111, Test Acc: 0.7259
Epoch: 020, Train Acc: 0.6522, Test Acc: 0.6716
Epoch: 021, Train Acc: 0.7280, Test Acc: